# Анализ базы данных средствами SQL

## Введение

В нашем распоряжении есть база данных крупного сервиса для чтения книг по подписке. Нам как аналитику необходимо проанализировать базу данных.

**Цель исследования:**
- Бизнесу необходимо сформулировать ценностное предложение для нового продукта.

Для достижения этой цели нам необходимо решить следующие задачи:

1. Посчитать, сколько книг вышло после 1 января 2000 года;
2. Для каждой книги посчитать количество обзоров и среднюю оценку;
3. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры;
4. Определить автора с самой высокой средней оценкой книг — учитывая только книги с 50 и более оценками;
5. Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

**Описание данных:**

                                                 ER диаграмма базы данных
<img src="https://habrastorage.org/webt/7x/qi/cu/7xqicu6cw_j_wsnzhcfy1_ydpyi.png" width="700" height="700"/>

1. Таблица `books`

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

2. Таблица `authors`

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

3. Таблица `publishers`

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

4. Таблица `ratings`

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

5. Таблица `reviews`

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

**Ход исследования:**

Первым делом приступим к обзору данных, проверим качество. Далее приступим к решению задач путем формирования SQL запросов к базе данных и опишем выводы по каждой из решенных задач.

- Исследуйте таблицы — выведите первые строки;
- Сделайте по одному SQL-запросу для решения каждого задания;
- Выведите результат каждого запроса в тетрадке;
- Опишите выводы по каждой из решённых задач.

Исходя из этого наш проект будет состоять из четырех частей:

1. Введение
2. Загрузка и исследование таблиц
3. Решение задач с выводами

## Загрузка и исследование таблиц

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine 

### Подключение к базе данных

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

### Исследование таблиц

#### Таблица books

In [3]:
# Формируем sql-запросы и загружаем таблицы
query = ''' SELECT *
            FROM books
            LIMIT 5
        '''
# выгрузку сохраняем в переменную
books = pd.io.sql.read_sql(query, con=engine, index_col='book_id')
books

,author_id,title,num_pages,publication_date,publisher_id
book_id,,,,,
1,546,'Salem's Lot,594,2005-11-01,93
2,465,1 000 Places to See Before You Die,992,2003-05-22,336
3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
5,125,1776,386,2006-07-04,268


- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

#### Таблица authors

In [4]:
# Формируем sql-запросы и загружаем таблицы
query = ''' SELECT *
            FROM authors
            LIMIT 5
        '''
# выгрузку сохраняем в переменную
authors = pd.io.sql.read_sql(query, con=engine, index_col='author_id')
authors

,author
author_id,
1,A.S. Byatt
2,Aesop/Laura Harris/Laura Gibbs
3,Agatha Christie
4,Alan Brennert
5,Alan Moore/David Lloyd


- `author_id` — идентификатор автора;
- `author` — имя автора.

#### Таблица publisher

In [5]:
# Формируем sql-запросы и загружаем таблицы
query = ''' SELECT *
            FROM publishers
            LIMIT 5
        '''
# выгрузку сохраняем в переменную
publishers = pd.io.sql.read_sql(query, con=engine, index_col='publisher_id')
publishers

,publisher
publisher_id,
1,Ace
2,Ace Book
3,Ace Books
4,Ace Hardcover
5,Addison Wesley Publishing Company


- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;


#### Таблица ratings

In [6]:
# Формируем sql-запросы и загружаем таблицы
query = ''' SELECT *
            FROM ratings
            LIMIT 5
        '''
# выгрузку сохраняем в переменную
ratings = pd.io.sql.read_sql(query, con=engine, index_col='rating_id')
ratings

,book_id,username,rating
rating_id,,,
1,1,ryanfranco,4
2,1,grantpatricia,2
3,1,brandtandrea,5
4,2,lorichen,3
5,2,mariokeller,2


- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

#### Таблица reviews

In [7]:
# Формируем sql-запросы и загружаем таблицы
query = ''' SELECT *
            FROM reviews
            
        '''
# выгрузку сохраняем в переменную
reviews = pd.io.sql.read_sql(query, con=engine, index_col='review_id')
reviews

,book_id,username,text
review_id,,,
1,1,brandtandrea,Mention society tell send professor analysis. ...
2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
4,3,johnsonamanda,Finally month interesting blue could nature cu...
5,3,scotttamara,Nation purpose heavy give wait song will. List...
...,...,...,...
2789,999,martinadam,Later hospital turn easy community. Fact same ...
2790,1000,wknight,Change lose answer close pressure. Spend so now.
2791,1000,carolrodriguez,Authority go who television entire hair guy po...


- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Решение задач

### Сколько книг вышло после 1 января 2000 года

In [9]:
# Формируем sql-запрос
query = ''' SELECT COUNT(title)
            FROM books
            WHERE publication_date >= '2000-01-01';
        '''
# выгрузку сохраняем в переменную
task_1 = pd.io.sql.read_sql(query, con=engine)
task_1

,count
0,821


Количество книг, которые вышли после 1 января 2000 года равно 821 книге.

###  Количество обзоров и средняя оценка каждой книги

In [10]:
# Формируем sql-запрос
query = ''' WITH
            revies_cnt AS (SELECT book_id, COUNT(book_id) AS reviews_cnt
                           FROM reviews
                           GROUP BY book_id),
                           
            avg_rating AS (SELECT book_id, AVG(rating) AS avg_rating, COUNT(rating) AS rating_cnt
                           FROM ratings
                           GROUP BY book_id)
            
            SELECT title, reviews_cnt, rating_cnt, avg_rating
            FROM books AS b
            LEFT JOIN revies_cnt ON b.book_id = revies_cnt.book_id
            LEFT JOIN avg_rating ON b.book_id = avg_rating.book_id
            ORDER BY rating_cnt DESC
        '''
# выгрузку сохраняем в переменную
task_2 = pd.io.sql.read_sql(query, con=engine)
task_2

,title,reviews_cnt,rating_cnt,avg_rating
0,Twilight (Twilight #1),7.0,160,3.662500
1,The Hobbit or There and Back Again,6.0,88,4.125000
2,The Catcher in the Rye,6.0,86,3.825581
3,Angels & Demons (Robert Langdon #1),5.0,84,3.678571
4,Harry Potter and the Prisoner of Azkaban (Harr...,6.0,82,4.414634
...,...,...,...,...
995,Angels Flight (Harry Bosch #6; Harry Bosch Un...,2.0,2,4.500000
996,The Big Bad Wolf (Alex Cross #9),2.0,2,5.000000
997,Amsterdam,2.0,2,2.500000
998,The Burden of Proof (Kindle County Legal Thril...,1.0,2,3.500000


Самое большое кол-во оценок приходится на Twilight (Сумерки), следом идет наш любимый Хоббит Толкина. Можем заметить, что не по всем книгам в базе присутствуют обзоры. Присутствуют книги с маленьким количеством оценок.

### Издательство, которое выпустило наибольшее число книг толще 50 страниц

In [11]:
# Формируем sql-запрос
query = ''' SELECT publisher, COUNT(title) AS title_cnt
            FROM books AS b
            LEFT JOIN publishers AS p ON b.publisher_id = p.publisher_id
            WHERE num_pages > 50
            GROUP BY publisher
            ORDER BY title_cnt DESC
            LIMIT 1
        '''
# выгрузку сохраняем в переменную
task_3 = pd.io.sql.read_sql(query, con=engine)
task_3

,publisher,title_cnt
0,Penguin Books,42


Издательство Penguin Books выпустило наибольшее кол-во книг (42 книги) толще 50 страниц.

### Автора с самой высокой средней оценкой книг — учитывая только книги с 50 и более оценками

In [12]:
query = ''' WITH
            books_over_50_cnt_rat AS (SELECT book_id, COUNT(rating) AS cnt_ratings
                                      FROM ratings AS rat
                                      GROUP BY book_id
                                      HAVING COUNT(rating) > 50)
                                      
            SELECT author, AVG(r.rating) AS mean_rating
            FROM books_over_50_cnt_rat
            INNER JOIN books AS b ON books_over_50_cnt_rat.book_id = b.book_id
            INNER JOIN authors AS a ON b.author_id = a.author_id
            INNER JOIN ratings AS r ON b.book_id = r.book_id
            GROUP BY author
            ORDER BY mean_rating DESC
        '''
# выгрузку сохраняем в переменную
task_4 = pd.io.sql.read_sql(query, con=engine)
task_4

,author,mean_rating
0,J.K. Rowling/Mary GrandPré,4.287097
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.246914
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Lois Lowry,3.750000


Автором с самой высокой средней оценкой книг является Джоан Роулинг, известная по серии книг "Гарри Поттер".

### Среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [13]:
# Формируем sql-запрос
query = ''' WITH
            rating_cnt AS (SELECT username, COUNT(rating) AS rating_cnt
                           FROM ratings
                           GROUP BY username),
                           
            reviews_cnt AS (SELECT COUNT(review_id) AS reviews_cnt_where_rating_cnt_over_50
                            FROM reviews
                            WHERE username IN (SELECT username
                                               FROM rating_cnt
                                               WHERE rating_cnt > 50)
                            GROUP BY username)
            
            SELECT ROUND(AVG(reviews_cnt_where_rating_cnt_over_50), 0)
            FROM reviews_cnt
        '''
# выгрузку сохраняем в переменную
task_5 = pd.io.sql.read_sql(query, con=engine)
task_5

,round
0,24.0


Среднее количество обзоров от пользователей, которые поставили больше 50 оценок равно 24.

## Вывод

В нашем распоряжении была база данных крупного сервиса для чтения книг по подписке. Нам как аналитику необходимо было проанализировать базу данных, чтобы помочь бизнесу сформулировать ценностное предложение для нового продукта.

**В процессе загрузки данных проверили соединение с сервером и выяснили содержание следующих таблиц:**
1. Таблица books содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

2. Таблица authors содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

3. Таблица publishers содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

4. Таблица ratings содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.


5. Таблица reviews содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

**Решили поставленные задачи и отметили следующее:**
- Количество книг, которые вышли после 1 января 2000 года равно 821 книге.
- Самое большое кол-во оценок приходится на Twilight (Сумерки). Отметили, что ни по всем книгам в базе присутствуют обзоры. Присутствуют книги с маленьким количеством оценок.
- Издательство Penguin Books выпустило наибольшее кол-во книг (42 книги) толще 50 страниц.
- Автором с самой высокой средней оценкой книг является Джоан Роулинг, известная по серии книг "Гарри Поттер".
- Среднее количество обзоров от пользователей, которые поставили больше 50 оценок равно 24.
